<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

<center>

#SemEval 2023 Shared Task 12: AfriSenti (Task A)

###Starter Notebook

</center>

Baseline code based on the starter code

#1) Installations and imports

##a. Mount drive (if you are running on colab)

##b. Clone or update competition repository

After cloning, under MyDrive, you will see afrisenti-semeval-2023 folder with all the the data for the afrisenti shared task (training and dev) 

In [1]:
import os

# Language to train sentiment classifier for
# am dz ha ig ma pcm pt sw yo

LANGUAGE_CODE = 'yo'
folder = ''

colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    proj_folder = '/content/drive/MyDrive'
else:
    proj_folder = os.getcwd()

In [2]:
%cd {proj_folder}


PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'
PROJECT_GITHUB_URL = 'https://github.com/afrisenti-semeval/afrisent-semeval-2023.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  pass
#   !git pull {PROJECT_GITHUB_URL}

C:\Users\Thomas\Downloads\afrisent
C:\Users\Thomas\Downloads\afrisent\afrisent-semeval-2023


##c. Install required libraries

- Set the project dire
ctory in the cell below, where the requirements file should also be located, and run the cell

In [3]:
%cd {PROJECT_DIR}

if os.path.isdir(PROJECT_DIR):
  #The requirements file should be in PROJECT_DIR
  if os.path.isfile(os.path.join(PROJECT_DIR, 'starter_kit/requirements.txt')):
    !pip install -r starter_kit/requirements.txt
  else:
    print('requirements.txt file not found')

else:
  print("Project directory not found, please check again.")

C:\Users\Thomas\Downloads\afrisent\afrisent-semeval-2023


##d. Import libraries

Import libraries below

In [4]:
import pandas as pd
import numpy as np

# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

#2) Dataset

##a. Formatting

The training dataset that was provided for the competition is in the following format:

| ID | text | label |
| --- | --- | --- |
| twt001 | example text | negative |
| twt002 | example text | positive |
| ... | ... | ... |

However, the code in the starter kit do not expect the 
ID and require the training (and evaluation) data to be in the following format

|text | label |
|--- | --- |
|example text | negative |
|example text | positive |
|... | ... |

To reformat the data run the following cell

In [5]:
# Training Data Paths

TASK = 'SubtaskA'
TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK, 'train')
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

if os.path.isdir(TRAINING_DATA_DIR):
  print('Data directory found.')
  if not os.path.isdir(FORMATTED_TRAIN_DATA):
    print('Creating directory to store formatted data.')
    os.mkdir(FORMATTED_TRAIN_DATA)
else:
  print(TRAINING_DATA_DIR + ' is not a valid directory or does not exist!')

Data directory found.


In [6]:
%cd {TRAINING_DATA_DIR}

training_files = os.listdir()

if len(training_files) > 0:
  for training_file in training_files:
    if training_file.endswith('.tsv'):

      data = training_file.split('_')[0]
      if not os.path.isdir(os.path.join(FORMATTED_TRAIN_DATA, data)):
        print(data, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(FORMATTED_TRAIN_DATA, data))
      
      df = pd.read_csv(training_file, sep='\t', names=['ID', 'text', 'label'], header=0)
      df[['text', 'label']].to_csv(os.path.join(FORMATTED_TRAIN_DATA, data, 'train.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' skipped!')
else:
  print('No files are found in this directory!')

C:\Users\Thomas\Downloads\afrisent\afrisent-semeval-2023\SubtaskA\train
formatted-train-data skipped!
README.txt skipped!
splitted-train-dev-test skipped!


After running the code above, a new folder (called formated-train-data) with formated files is created in the "datasets" folder in the train sub-folder.

##b. <font color='red'>`(Optional) Creating Evaluation (Dev and Test) sets from the available training data`</font>

You may wish to create train and evaluation (dev and test) sets from the training data provided. If you wish to do so, you can run any of the cells below`

###i. If you want to create both the Dev and Test sets, run this cell

In [7]:
if os.path.isdir(FORMATTED_TRAIN_DATA):
  print('Data directory found.')
  SPLITTED_DATA = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test')
  if not os.path.isdir(SPLITTED_DATA):
    print('Creating directory to store train, dev and test splits.')
    os.mkdir(SPLITTED_DATA)
else:
  print(FORMATTED_TRAIN_DATA + ' is not a valid directory or does not exist!')

%cd {FORMATTED_TRAIN_DATA}
formatted_training_files = os.listdir()

if len(formatted_training_files) > 0:
  for data_name in formatted_training_files:
    formatted_training_file = os.path.join(data_name, 'train.tsv')
    if os.path.isfile(formatted_training_file):
      labeled_tweets = pd.read_csv(formatted_training_file, sep='\t', names=['text', 'label'], header=0)
      train, dev, test = np.split(labeled_tweets.sample(frac=1, random_state=42), [int(.7*len(labeled_tweets)), int(.8*len(labeled_tweets))])

      if not os.path.isdir(os.path.join(SPLITTED_DATA, data_name)):
        print(data_name, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(SPLITTED_DATA, data_name))

      train.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'train.tsv'), sep='\t', index=False)
      dev.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'dev.tsv'), sep='\t', index=False)
      test.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name,'test.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' is not a supported file!')
else:
  print('No files are found in this directory!')

Data directory found.
C:\Users\Thomas\Downloads\afrisent\afrisent-semeval-2023\SubtaskA\train\formatted-train-data


#3) Training setup

##a. Set project parameters

For a list of models that be used for fine-tuning, you can check [HERE](https://huggingface.co/models).

In [8]:
%cd {PROJECT_DIR}

if LANGUAGE_CODE in languages:
  # Model Training Parameters
  MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
  BATCH_SIZE = 32
  LEARNING_RATE = 5e-5
  NUMBER_OF_TRAINING_EPOCHS = 5
  MAXIMUM_SEQUENCE_LENGTH = 128
  SAVE_STEPS = -1

  print('Everything set. You can now start model training.')

else:
  print("Invalid language code/Dataset not released. Please input any of the following released data\n\n\t- 'am'\n\t- 'dz'\n\t- 'ha'\n\t- 'ig'\n\t- 'ma'\n\t- 'pcm'\n\t- 'pt'\n\t- 'sw'\n\t- 'yo'")

C:\Users\Thomas\Downloads\afrisent\afrisent-semeval-2023
Everything set. You can now start model training.


##b. Train the model

In the section below, we provide three options: 

- 1) training model without any validation; 
- 2) training model with validation but without testing; 
- 3) training a model with validation and test set.

###Model Training



####Starter Code: Datasets, etc


In [9]:
from types import SimpleNamespace
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
import pandas
import pandas as pd
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from datasets import Features, Value, ClassLabel, load_dataset, Dataset

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

logger = logging.getLogger(__name__)


with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")


np.random.seed(420)
torch.manual_seed(69)

NOTE: Redirects are currently not supported in Windows or MacOs.


In [10]:
TRAINING_DATA_DIR

'C:\\Users\\Thomas\\Downloads\\afrisent/afrisent-semeval-2023\\SubtaskA\\train'

In [11]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
EVAL_DIR = os.path.join(PROJECT_DIR, TASK, 'dev')
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_eval')

training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUMBER_OF_TRAINING_EPOCHS,
    output_dir=OUTPUT_DIR,
    save_steps=SAVE_STEPS,
    overwrite_output_dir=True
    )

In [12]:
data_args = SimpleNamespace(**{
    'max_seq_length': MAXIMUM_SEQUENCE_LENGTH,
    'overwrite_cache': False,
    'pad_to_max_length': True,
    'max_train_samples': None,
    'max_eval_samples': None,
    'max_predict_samples': None
})

model_args = SimpleNamespace(**{
    'model_name_or_path': MODEL_NAME_OR_PATH,
    'config_name': None,
    'tokenizer_name': None,
    'data_dir': DATA_DIR,
    'eval_dir': EVAL_DIR,
    'cache_dir': None,
    'do_lower_case': None,
    'use_fast_tokenizer': True,
    'model_revision': 'main',
    'use_auth_token': False,
    'ignore_mismatched_sizes': False
})


In [13]:
# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.

# parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
# model_args, data_args, training_args = parser.parse_args_into_dataclasses()

# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
#send_example_telemetry("run_xnli", model_args)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = 0 #training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

In [14]:
# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

10/30/2022 04:22:41 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False


In [15]:
LANGUAGE_CODE

'yo'

In [16]:
# Set seed before initializing model.
set_seed(training_args.seed)

# In distributed training, the load_dataset function guarantees that only one local process can concurrently
# download the dataset.
# Downloading and loading xnli dataset from the hub.


if training_args.do_train:

    #train_dataset = load_dataset('csv', data_files={'train': model_args.data_dir + '/train.csv'}, cache_dir=model_args.cache_dir)
    #df = train_dataset["train"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = train_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/train.tsv', sep='\t')
    df = df.dropna()
    train_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

if training_args.do_eval:
    #eval_dataset = load_dataset('csv', data_files={'validation': model_args.data_dir + '/dev.csv'}, cache_dir=model_args.cache_dir)

    #df = eval_dataset["validation"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = eval_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/dev.tsv', sep='\t')
    df = df.dropna()
    eval_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

if training_args.do_predict:
    #predict_dataset = load_dataset('csv', data_files={'test': model_args.data_dir + '/test.csv'}, cache_dir=model_args.cache_dir)

    #df = predict_dataset["test"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = predict_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/test.tsv', sep='\t')
    df = df.dropna()
    predict_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

# Labels
num_labels = len(label_list)
print(label_list)

['positive', 'neutral', 'negative']


####Tokenization

In [17]:
# Load pretrained model and tokenizer
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
#         finetuning_task="xnli",
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    do_lower_case=model_args.do_lower_case,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
    ignore_mismatched_sizes=model_args.ignore_mismatched_sizes,
)

[WARNING|modeling_utils.py:2596] 2022-10-30 04:22:42,741 >> Some weights of the model checkpoint at Davlan/afro-xlmr-mini were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[WARNING|modeling_utils.py:2608] 2022-10-30 04:22:42,742 >> Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-mini a

In [18]:
# Preprocessing the datasets
# Padding strategy
if data_args.pad_to_max_length:
    padding = "max_length"
else:
    # We will pad later, dynamically at batch creation, to the max sequence length in each batch
    padding = False

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
label_to_id = {v: i for i, v in enumerate(label_list)}

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

'''
    def preprocess_function(examples):
    # Tokenize the texts
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        padding=padding,
        max_length=data_args.max_seq_length,
        truncation=True,
    )
'''

'\n    def preprocess_function(examples):\n    # Tokenize the texts\n    return tokenizer(\n        examples["premise"],\n        examples["hypothesis"],\n        padding=padding,\n        max_length=data_args.max_seq_length,\n        truncation=True,\n    )\n'

In [19]:
def preprocess_function(examples):
    # Tokenize the texts
    #print
    texts =(examples['text'],)
    result = tokenizer(*texts, padding=padding, max_length=data_args.max_seq_length, truncation=True)
    #print(examples['text'])
    #result = tokenizer(examples['text'], examples['text'], padding=padding, max_length=data_args.max_seq_length, truncation=True)
    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    
    result['length'], result["tokenized"] = [], []
    for input_ids in result['input_ids']:
        toks = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        result['length'].append(len(toks)+2)
        result['tokenized'].append(' '.join(toks))
    return result

if training_args.do_train:
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))
    with training_args.main_process_first(desc="train dataset map pre-processing"):
        train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on train dataset",
        )
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

if training_args.do_eval:
    if data_args.max_eval_samples is not None:
        max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
        eval_dataset = eval_dataset.select(range(max_eval_samples))
    with training_args.main_process_first(desc="validation dataset map pre-processing"):
        eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on validation dataset",
        )

if training_args.do_predict:
    if data_args.max_predict_samples is not None:
        max_predict_samples = min(len(predict_dataset), data_args.max_predict_samples)
        predict_dataset = predict_dataset.select(range(max_predict_samples))
    with training_args.main_process_first(desc="prediction dataset map pre-processing"):
        predict_dataset = predict_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on prediction dataset",
        )

Running tokenizer on train dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
train_dataset, eval_dataset

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 5965
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 852
 }))

####LSTM Baseline

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [22]:
class LSTM(nn.Module):
    def __init__(self, hidden_dim=128, emb_dim=300, num_layers=1, dropout=0.5, lstm_dropout=0.0):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(len(tokenizer), emb_dim)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size=emb_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=lstm_dropout)
        
        self.drop = nn.Dropout(p=dropout)

        self.fc = nn.Linear(2*hidden_dim, 3)

    def forward(self, text, text_len):

        text_emb = self.embedding(text)

        packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.hidden_dim]
        out_reverse = output[:, 0, self.hidden_dim:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = text_fea
        return text_out

In [23]:
num_pts = len(train_dataset)
shuffled_ids = np.arange(num_pts, dtype=int)
np.random.shuffle(shuffled_ids)

valid_ids = torch.LongTensor(np.array(train_dataset['input_ids'])[shuffled_ids[:num_pts // 5]])
valid_lengths = torch.LongTensor(np.array(train_dataset['length'])[shuffled_ids[:num_pts // 5]]).cpu()
valid_labels = torch.LongTensor(np.array(train_dataset['label'])[shuffled_ids[:num_pts // 5]])

train_ids = torch.LongTensor(np.array(train_dataset['input_ids'])[shuffled_ids[num_pts // 5:]])
train_lengths = torch.LongTensor(np.array(train_dataset['length'])[shuffled_ids[num_pts // 5:]]).cpu()
train_labels = torch.LongTensor(np.array(train_dataset['label'])[shuffled_ids[num_pts // 5:]])

eval_ids = torch.LongTensor(eval_dataset['input_ids'])
eval_lengths = torch.LongTensor(eval_dataset['length']).cpu()
eval_labels = torch.LongTensor(eval_dataset['label'])
len(valid_ids), len(train_ids), len(eval_ids)

(1193, 4772, 852)

In [24]:
def train(criterion = nn.CrossEntropyLoss(),
          batch_size = 32,
          num_epochs = 30,
          eval_every = 4,
          params={},
          lr=0.0005,
          leave=False):
    model = LSTM(**params).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    eval_every = (len(train_ids) / batch_size) // eval_every
    best_valid_acc = 0
    best_valid_f1 = 0
    model.train()
    best_preds = []
    for epoch in range(num_epochs):
        pbar = tqdm(range(0, len(train_ids), batch_size), leave=leave, desc=f'Epoch {epoch+1}/{num_epochs}')
        total_train_loss = 0.0
        total_points = 0
        for i in pbar:           
            labels = train_labels[i:i+batch_size].to(device)
            inps = train_ids[i:i+batch_size].to(device)
            lengths = train_lengths[i:i+batch_size]#.to(device)
            output = model(inps, lengths)

            loss = criterion(output, labels)
            total_train_loss += loss.item()
            total_points += labels.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + batch_size) % eval_every == 0:
                model.eval()
                with torch.no_grad():
                    num_correct = 0
                    eval_batch_size = 100
                    for j in range(0, len(valid_ids), eval_batch_size):
                        labels = valid_labels[j:j+eval_batch_size].to(device)
                        inps = valid_ids[j:j+eval_batch_size].to(device)
                        lengths = valid_lengths[j:j+eval_batch_size]#.to(device)
                        output = model(inps, lengths)
                        output = torch.argmax(output, -1)
                        num_correct += torch.sum(output == labels).cpu().numpy()
                    valid_accuracy = num_correct / len(valid_ids)

                    accuracy = "N/A"
                    preds = np.array([])
                    if valid_accuracy > best_valid_acc:
                        num_correct = 0
                        eval_batch_size = 100
                        for j in range(0, len(eval_ids), eval_batch_size):
                            labels = eval_labels[j:j+eval_batch_size].to(device)
                            inps = eval_ids[j:j+eval_batch_size].to(device)
                            lengths = eval_lengths[j:j+eval_batch_size]#.to(device)
                            output = model(inps, lengths)
                            output = torch.argmax(output, -1)
                            preds = np.append(preds, output.cpu().numpy())
                            num_correct += torch.sum(output == labels).cpu().numpy()
                        accuracy = num_correct / len(eval_ids)
                        best_valid_f1 = f1_score(preds, eval_labels, average='weighted')
                        best_preds = preds
                        best_valid_acc = accuracy
                    pbar.set_description(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {(total_train_loss / total_points):.3} ' + \
                                         f'Val Acc: {valid_accuracy:.3}, Eval Acc: {accuracy:.3}, Eval Acc @ Best Val {best_valid_acc:.3}, Eval F1: {best_valid_f1:.3}')
                model.train()
    return best_valid_acc, best_valid_f1, preds, eval_labels




acc, f1, preds, labels = train(num_epochs=15, params={
    'hidden_dim': 128, 'emb_dim': 300, 'num_layers': 2, 'dropout': 0.0, 'lstm_dropout': 0.5},
      lr=0.001, leave=True)

Epoch 1/15, Train Loss: 0.0312 Val Acc: 0.583, Eval Acc: 0.526, Eval Acc @ Best Val 0.526, Eval F1: 0.566: 100%|█| 150/
Epoch 2/15, Train Loss: 0.0253 Val Acc: 0.597, Eval Acc: 0.56, Eval Acc @ Best Val 0.56, Eval F1: 0.575: 100%|█| 150/15
Epoch 3/15, Train Loss: 0.0192 Val Acc: 0.598, Eval Acc: 0.581, Eval Acc @ Best Val 0.581, Eval F1: 0.585: 100%|█| 150/
Epoch 4/15, Train Loss: 0.0128 Val Acc: 0.605, Eval Acc: 0.575, Eval Acc @ Best Val 0.575, Eval F1: 0.574: 100%|█| 150/
Epoch 5/15, Train Loss: 0.00816 Val Acc: 0.593, Eval Acc: 0.574, Eval Acc @ Best Val 0.574, Eval F1: 0.572: 100%|█| 150
Epoch 6/15, Train Loss: 0.00537 Val Acc: 0.617, Eval Acc: 0.587, Eval Acc @ Best Val 0.587, Eval F1: 0.595: 100%|█| 150
Epoch 7/15, Train Loss: 0.00377 Val Acc: 0.602, Eval Acc: 0.57, Eval Acc @ Best Val 0.57, Eval F1: 0.569: 100%|█| 150/1
Epoch 8/15, Train Loss: 0.00339 Val Acc: 0.616, Eval Acc: 0.583, Eval Acc @ Best Val 0.583, Eval F1: 0.586: 100%|█| 150
Epoch 9/15, Train Loss: 0.00213 Val Acc:

In [25]:
del train_ids
del train_lengths
del train_labels

del eval_ids
del eval_lengths
del eval_labels


####Pretrained Transformer Baseline - xlmr

In [26]:
# Get the metric function
metric = evaluate.load("accuracy")

# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Data collator will default to DataCollatorWithPadding, so we change it if we already did the padding.
if data_args.pad_to_max_length:
    data_collator = default_data_collator
elif training_args.fp16:
    data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
else:
    data_collator = None

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    metrics = train_result.metrics
    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

# Evaluation
if training_args.do_eval or True:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate(eval_dataset=eval_dataset)

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
    
    
    splitted_A = os.path.join(PROJECT_DIR, 'SubtaskA', 'train', 'splitted-train-dev-test')
    
    try:
        LANGUAGE_CODE
    except NameError:
        LANGUAGE_CODE = 'combined'
    else:
        pass
    
    data = []
    for lang in languages:
        eval_path = os.path.join(splitted_A, lang)
        df = pd.read_csv(eval_path + '/dev.tsv', sep='\t')
        df = df.dropna()
        lang_eval = Dataset.from_pandas(df)
        lang_eval = lang_eval.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on validation dataset",
        )
        
        predictions, labels, metrics = trainer.predict(lang_eval, metric_key_prefix="eval")
        
        if LANGUAGE_CODE == lang:
            print(f1_score(np.argmax(predictions, axis=1), labels, average='weighted'))
        
        data.append([LANGUAGE_CODE, lang, str(list(predictions)), str(list(labels))])
    df = pd.DataFrame(data, columns=['source', 'target', 'predictions', 'labels'])
    df.to_csv(f'{LANGUAGE_CODE}_preds.csv', index=False)


    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

# Prediction
if training_args.do_predict:
    logger.info("*** Predict ***")
    predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")

    max_predict_samples = (
        data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
    )
    metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

    trainer.log_metrics("predict", metrics)
    trainer.save_metrics("predict", metrics)

    predictions = np.argmax(predictions, axis=1)
    output_predict_file = os.path.join(training_args.output_dir, "predictions.txt")
    if trainer.is_world_process_zero():
        with open(output_predict_file, "w") as writer:
            writer.write("index\tprediction\n")
            for index, item in enumerate(predictions):
                item = label_list[item]
                writer.write(f"{index}\t{item}\n")

[INFO|trainer.py:725] 2022-10-30 04:24:34,180 >> The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, tokenized, length. If text, tokenized, length are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
[INFO|trainer.py:1607] 2022-10-30 04:24:34,191 >> ***** Running training *****
[INFO|trainer.py:1608] 2022-10-30 04:24:34,192 >>   Num examples = 5965
[INFO|trainer.py:1609] 2022-10-30 04:24:34,192 >>   Num Epochs = 5
[INFO|trainer.py:1610] 2022-10-30 04:24:34,192 >>   Instantaneous batch size per device = 32
[INFO|trainer.py:1611

Step,Training Loss
500,0.903900


[INFO|trainer.py:1852] 2022-10-30 04:26:58,435 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:2656] 2022-10-30 04:26:58,453 >> Saving model checkpoint to C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_eval
[INFO|configuration_utils.py:447] 2022-10-30 04:26:58,455 >> Configuration saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_eval\config.json
[INFO|modeling_utils.py:1624] 2022-10-30 04:26:59,166 >> Model weights saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_eval\pytorch_model.bin
[INFO|tokenization_utils_base.py:2123] 2022-10-30 04:26:59,167 >> tokenizer config file saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_eval\tokenizer_config.json
[INFO|tokenization_utils_base.py:2130] 2022-10-30 04:26:59,168 >> Special tokens file saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_eval\spec

***** train metrics *****
  epoch                    =        5.0
  total_flos               =   457439GF
  train_loss               =     0.8028
  train_runtime            = 0:02:24.25
  train_samples            =       5965
  train_samples_per_second =    206.746
  train_steps_per_second   =      6.481


Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

[INFO|trainer.py:725] 2022-10-30 04:27:19,115 >> The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, tokenized, length. If text, tokenized, length are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2907] 2022-10-30 04:27:19,117 >> ***** Running Prediction *****
[INFO|trainer.py:2909] 2022-10-30 04:27:19,117 >>   Num examples = 181
[INFO|trainer.py:2912] 2022-10-30 04:27:19,118 >>   Batch size = 8


Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

[INFO|trainer.py:725] 2022-10-30 04:27:21,217 >> The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, tokenized, length. If text, tokenized, length are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2907] 2022-10-30 04:27:21,219 >> ***** Running Prediction *****
[INFO|trainer.py:2909] 2022-10-30 04:27:21,219 >>   Num examples = 852
[INFO|trainer.py:2912] 2022-10-30 04:27:21,219 >>   Batch size = 8


0.6754704067024604
***** eval metrics *****
  eval_accuracy           =     0.6737
  eval_loss               =     0.7884
  eval_runtime            = 0:00:01.53
  eval_samples_per_second =    556.602
  eval_steps_per_second   =     69.902


#### KinyaBert

In [27]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_test')
kinya = 'jean-paul/KinyaBERT-small'

!python starter_kit/run_textclass.py \
  --model_name_or_path {kinya} \
  --data_dir {DATA_DIR} \
  --do_train \
  --do_eval \
  --per_device_train_batch_size {BATCH_SIZE} \
  --learning_rate {MAXIMUM_SEQUENCE_LENGTH} \
  --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
  --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
  --output_dir {OUTPUT_DIR} \
  --save_steps {SAVE_STEPS} \
  --overwrite_output_dir

10/30/2022 04:27:27 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
10/30/2022 04:27:27 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
h

NOTE: Redirects are currently not supported in Windows or MacOs.
[INFO|configuration_utils.py:653] 2022-10-30 04:27:27,296 >> loading configuration file config.json from cache at C:\Users\Thomas/.cache\huggingface\hub\models--jean-paul--KinyaBERT-small\snapshots\4575f11375376ae29a8b19610f70c07ee04d02eb\config.json
[INFO|configuration_utils.py:705] 2022-10-30 04:27:27,300 >> Model config BertConfig {
  "_name_or_path": "jean-paul/KinyaBERT-small",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads":

100%|#########9| 934/935 [02:46<00:00,  5.66it/s][INFO|trainer.py:1852] 2022-10-30 04:30:15,209 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)



                                                 

100%|##########| 935/935 [02:46<00:00,  5.63it/s]
[INFO|trainer.py:2656] 2022-10-30 04:30:15,220 >> Saving model checkpoint to C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_test
[INFO|configuration_utils.py:447] 2022-10-30 04:30:15,222 >> Configuration saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_test\config.json
[INFO|modeling_utils.py:1624] 2022-10-30 04:30:15,572 >> Model weights saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_test\pytorch_model.bin
[INFO|tokenization_utils_base.py:2123] 2022-10-30 04:30:15,573 >> tokenizer config file saved in C:\Users\Thomas\Downloads\afrisent/afrisent-semeval-2023\models\yo_no_test\tokenizer_config.json
[INFO|tokenization_util